In [4]:
import nest_asyncio
import os

os.chdir("/home/carnufex/butlerbot")

nest_asyncio.apply()

In [7]:
from news_grabber import get_personalized_news

get_personalized_news()

ModuleNotFoundError: No module named 'news_grabber'

In [20]:
from newsapi import NewsApiClient
from typing import List, Dict
from datetime import datetime, timedelta
from src.settings import settings


class NewsFetcher:
    def __init__(self, api_key: str):
        """Initialize NewsAPI client with your API key."""
        self.api = NewsApiClient(api_key=api_key)

    def get_personalized_news(
        self,
        topics: List[str],
        preferred_sources: List[str] = None,
        days_back: int = 7,
        language: str = "en",
    ) -> List[Dict]:
        """
        Fetch news based on personal preferences.

        Args:
            topics: List of topics/keywords to search for
            preferred_sources: List of preferred news sources (domains)
            days_back: How many days back to search
            language: Language of news articles

        Returns:
            List of news articles matching criteria
        """
        # Calculate date range
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days_back)

        all_articles = []

        # Search for each topic
        for topic in topics:
            query = topic

            # Add source filtering if specified
            if preferred_sources:
                domains = ",".join(preferred_sources)
                query = f"{query} AND domains:{domains}"

            try:
                response = self.api.get_everything(
                    q=query,
                    from_param=start_date.strftime("%Y-%m-%d"),
                    to=end_date.strftime("%Y-%m-%d"),
                    language=language,
                    sort_by="relevancy",
                )

                if response["status"] == "ok":
                    all_articles.extend(response["articles"])

            except Exception as e:
                print(f"Error fetching news for topic '{topic}': {str(e)}")

        # Remove duplicates based on article URL
        unique_articles = {article["url"]: article for article in all_articles}
        return list(unique_articles.values())

    def get_top_headlines(
        self,
        country: str = None,
        category: str = None,
        sources: List[str] = None,
        language: str = "en",
    ) -> List[Dict]:
        """
        Fetch top headlines for a specific country, category, or sources.

        Args:
            country: Country code (e.g., 'us', 'gb') for headlines
            category: News category (e.g., 'business', 'sports')
            sources: List of specific news sources
            language: Language of news articles

        Returns:
            List of top headlines
        """
        try:
            response = self.api.get_top_headlines(
                country=country,
                category=category,
                sources=",".join(sources) if sources else None,
                language=language,
            )

            if response["status"] == "ok":
                return response["articles"]
            else:
                print(
                    f"Error fetching top headlines: {response.get('message', 'Unknown error')}"
                )
                return []

        except Exception as e:
            print(f"Error fetching top headlines: {str(e)}")
            return []


# Example usage
def get_personalized_news():
    """Fetch personalized news articles based on user preferences."""

    # Initialize the news fetcher
    news_fetcher = NewsFetcher(settings.NEWS_API_KEY)

    # Fetch personalized news
    return news_fetcher.get_personalized_news(
        topics=settings.NEWS_TOPICS,
        preferred_sources=settings.NEWS_SOURCES,
        days_back=1,
    )


def get_top_us_news():
    """Fetch top news headlines from the US."""

    # Initialize the news fetcher
    news_fetcher = NewsFetcher(settings.NEWS_API_KEY)

    # Fetch top US news
    return news_fetcher.get_top_headlines(
        country="us",
    )

In [14]:
news_fetcher = NewsFetcher(settings.NEWS_API_KEY)

In [ ]:
def get_top_us_news_by_category():
    """Fetch top news headlines from the US for specific categories."""

    # Initialize the news fetcher
    news_fetcher = NewsFetcher(settings.NEWS_API_KEY)

    categories = [
        "business",
        "entertainment",
        "general",
        "health",
        "science",
        "sports",
        "technology",
    ]
    news_by_category = {}

    for category in categories:
        news_by_category[category] = news_fetcher.get_top_headlines(
            country="us",
            category=category,
        )

    return news_by_category


get_top_us_news_by_category()

{'business': [{'source': {'id': 'associated-press',
    'name': 'Associated Press'},
   'author': 'Aamer Madhani, Christopher Rugaber, Josh Boak',
   'title': "Trump suggests he can remove Fed Chair Powell and says he's 'not happy' with him over interest rates - AP News",
   'description': 'President Donald Trump has attacked Federal Reserve Chair Jerome Powell for not cutting interest rates. And Trump says he could fire Powell if he wanted to. Trump is renewing a threat from his first term and it could cause a major legal showdown over the cent…',
   'url': 'https://apnews.com/article/trump-powell-federal-reserve-fed-termination-b6148c8048dda538a6ca3b5a270fd09e',
   'urlToImage': 'https://dims.apnews.com/dims4/default/7ccb842/2147483647/strip/true/crop/3447x1939+0+183/resize/1440x810!/quality/90/?url=https%3A%2F%2Fassets.apnews.com%2F8c%2F62%2Feac00ffe42b668f64dbb1587c708%2Fd6484aa6d741473cad05519a2159b675',
   'publishedAt': '2025-04-17T22:29:00Z',
   'content': 'WASHINGTON (AP) Pres

: 